In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from docx import Document
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer
import json
import os
import gc


In [3]:
df = pd.read_csv("/home/macab/research/BloomNet/data/BloomTaxonomy.csv")

In [74]:
df['Label'].value_counts()

Comprehension    100
Knowledge        100
Application      100
Analysis         100
Synthesis        100
Evaluation       100
Name: Label, dtype: int64

In [76]:
df[20:25]

,Text,Label
20,How much water does an elephant drink,Knowledge
21,Identify a phrase as a simile,Knowledge
22,"Identify a property (commutative, additive, etc)",Knowledge
23,Identify fractions from a pictorial representa...,Knowledge
24,Identify ordinal positions,Knowledge


In [58]:
## create a label2target and target2label dictionary
encoder = LabelEncoder().fit(df['Label'])

In [59]:
encoder.classes_

array(['Analysis', 'Application', 'Comprehension', 'Evaluation',
       'Knowledge', 'Synthesis'], dtype=object)

In [60]:
label2idx  = dict(zip(encoder.classes_, list(range(0, 6, 1))))

In [65]:
idx2label = dict(zip(list(range(0, 6, 1)), encoder.classes_))

In [66]:
print(label2idx)

{'Analysis': 0, 'Application': 1, 'Comprehension': 2, 'Evaluation': 3, 'Knowledge': 4, 'Synthesis': 5}


In [67]:
print(idx2label)

{0: 'Analysis', 1: 'Application', 2: 'Comprehension', 3: 'Evaluation', 4: 'Knowledge', 5: 'Synthesis'}


In [68]:
with open("./label-dictionaries/label2idx.json", "w") as file:
    json.dump(label2idx, file)
with open("./label-dictionaries/idx2label.json", "w") as file:
    json.dump(idx2label, file)

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [8]:
max(df['Text'].apply(lambda x:len(tokenizer.tokenize(x))))

44

In [9]:
df['Label'].value_counts()

Evaluation       100
Application      100
Analysis         100
Knowledge        100
Synthesis        100
Comprehension    100
Name: Label, dtype: int64

In [12]:
df = pd.read_csv("/home/macab/research/BloomNet/data/BloomTaxonomy2.csv")

In [15]:
sum(df['Label'].value_counts())

141

In [1]:
from dataset.dataset import BloomDataset
from transformers import AutoTokenizer
from config import config
import torch
from sklearn.model_selection import StratifiedKFold, KFold
import torch.nn as nn
from models.baselines.LSTM import LSTMClassifier
from dataset.loader import get_loaders

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [3]:
dataset = BloomDataset(
    tokenizer=tokenizer, 
    file_name="/home/macab/research/BloomNet/data/BloomTaxonomy.csv", 
    text_field=config['data']['Text'], 
    label_field=config['data']['Label'], 
    max_len=config['data']['max_len']
)

Using custom data configuration default-f2a03dde2a5e9193
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-f2a03dde2a5e9193/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [4]:
# dataset[0]

In [5]:
kfold = KFold(n_splits=4, shuffle=True, random_state=42)

In [6]:
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

      # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    

    print(f'fold = {fold} | trainlaoder = {len(trainloader)*10} | testloader = {len(testloader)*10}')

FOLD 0
--------------------------------
fold = 0 | trainlaoder = 450 | testloader = 150
FOLD 1
--------------------------------
fold = 1 | trainlaoder = 450 | testloader = 150
FOLD 2
--------------------------------
fold = 2 | trainlaoder = 450 | testloader = 150
FOLD 3
--------------------------------
fold = 3 | trainlaoder = 450 | testloader = 150


In [7]:
dataset2 = BloomDataset(
    tokenizer=tokenizer, 
    file_name="/home/macab/research/BloomNet/data/BloomTaxonomy2.csv", 
    text_field=config['data']['Text'], 
    label_field=config['data']['Label'], 
    max_len=config['data']['max_len']
)

Using custom data configuration default-9fbfd2cb2fcd0f38
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-9fbfd2cb2fcd0f38/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [8]:
batch = next(iter(trainloader))

In [9]:
batch['input_ids'].shape

torch.Size([10, 64])

In [10]:
model = LSTMClassifier(
    batch_size=10,
    output_size=6,
    hidden_size=768,
    vocab_size=tokenizer.vocab_size,
    embedding_length=768,
)

In [12]:
logits = model.forward(input_sentence=batch['input_ids'])

RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cpu and hidden tensor at cuda:0

In [1]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [2]:
pack_padded_sequence?

Signature:
pack_padded_sequence(
    input,
    lengths,
    batch_first=False,
    enforce_sorted=True,
)
Docstring:
Packs a Tensor containing padded sequences of variable length.

:attr:`input` can be of size ``T x B x *`` where `T` is the length of the
longest sequence (equal to ``lengths[0]``), ``B`` is the batch size, and
``*`` is any number of dimensions (including 0). If ``batch_first`` is
``True``, ``B x T x *`` :attr:`input` is expected.

For unsorted sequences, use `enforce_sorted = False`. If :attr:`enforce_sorted` is
``True``, the sequences should be sorted by length in a decreasing order, i.e.
``input[:,0]`` should be the longest sequence, and ``input[:,B-1]`` the shortest
one. `enforce_sorted = True` is only necessary for ONNX export.

Note:
    This function accepts any input that has at least two dimensions. You
    can apply it to pack the labels, and use the output of the RNN with
    them to compute the loss directly. A Tensor can be retrieved from
    a :class:`Pack